In [4]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

In [55]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks.head 

<bound method NDFrame.head of     Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]>

In [56]:
from sercret import key

In [57]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={key}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'BA(QAN CTGSLAO/)RTN ELDGSMLESKAE ', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'lciaffoi', 'close': None, 'closeTime': None, 'closeSource': 'alicfofi', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': 1689611299907, 'lowSource': 'rtlcepierIimX aE  e', 'latestPrice': 129.85, 'latestSource': 'Close', 'latestTime': 'January 15, 2021', 'latestUpdate': 1665935493686, 'latestVolume': None, 'iexRealtimePrice': 132.1, 'iexRealtimeSize': 68, 'iexLastUpdated': 1621417002838, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 129.41, 'previousVolume': 90516949, 'change': -1.83, 'changePercent': -0.0144, 'volume': None, 'iexMarketPercent': 0.0011175325612731823, 'iexVolume': 1893, 'avgTotalVolume':

In [58]:
price=data['latestPrice']
market_cap = data['marketCap']
market_cap/1000000000000

2.177664636542

In [30]:
my_clms=['Ticker','Stock Price','Market Capitalization', 'No of shares to buy']
my_df=pd.DataFrame(columns=my_clms)
my_df

,Ticker,Stock Price,Market Capitalization,No of shares to buy


In [36]:
my_df.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_clms
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,No of shares to buy
0,AAPL,128.94,2181174803877,N/A


In [38]:
my_df=pd.DataFrame(columns =my_clms)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={key}'
    data = requests.get(api_url).json()
    my_df=my_df.append(
        pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
        
    ],
        index=my_clms
    ),
    ignore_index=True
    )

In [39]:
my_df

,Ticker,Stock Price,Market Capitalization,No of shares to buy
0,A,133.02,2163868599875,N/A
1,AAL,132.58,2169150510053,N/A
2,AAP,130.08,2217676277407,N/A
3,AAPL,129.02,2176794960325,N/A
4,ABBV,129.06,2149815691793,N/A
...,...,...,...,...
500,YUM,132.37,2177160088697,N/A
501,ZBH,130.64,2243161078586,N/A
502,ZBRA,132.61,2208905889320,N/A
503,ZION,129.67,2175789397069,N/A


In [52]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:6587361


In [66]:
my_df=my_df.drop(["Number Of Shares to Buy","No Of Shares to buy"],axis=1)

In [67]:
my_df

,Ticker,Stock Price,Market Capitalization,No of shares to buy
0,A,133.02,2163868599875,N/A
1,AAL,132.58,2169150510053,N/A
2,AAP,130.08,2217676277407,N/A
3,AAPL,129.02,2176794960325,N/A
4,ABBV,129.06,2149815691793,N/A
...,...,...,...,...
500,YUM,132.37,2177160088697,N/A
501,ZBH,130.64,2243161078586,N/A
502,ZBRA,132.61,2208905889320,N/A
503,ZION,129.67,2175789397069,N/A


In [68]:
investment_amount = input("Enter total amount need to invest:")

try:
    val = float(portfolio_size)
except ValueError:
    print("It's not look like a number! \n Please try again:")
    portfolio_size = input("Enter total amount need to invest:")

Enter total amount need to invest:6880000


In [69]:
position_size = float(investment_amount) / len(my_df.index)
for i in range(0, len(my_df['Ticker'])-1):
    my_df.loc[i, 'No of shares to buy'] = math.floor(position_size / my_df['Stock Price'][i])
my_df

,Ticker,Stock Price,Market Capitalization,No of shares to buy
0,A,133.02,2163868599875,102
1,AAL,132.58,2169150510053,102
2,AAP,130.08,2217676277407,104
3,AAPL,129.02,2176794960325,105
4,ABBV,129.06,2149815691793,105
...,...,...,...,...
500,YUM,132.37,2177160088697,102
501,ZBH,130.64,2243161078586,104
502,ZBRA,132.61,2208905889320,102
503,ZION,129.67,2175789397069,105


In [71]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
my_df.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [72]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [73]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [74]:
writer.save()